# 뉴럴 네트워크 학습 알고리즘 구현

- tensorflow 2.0에서 지원하는 표준적인 방법 이용
- tensorflow 2.0은 크게 2가지로 나눠서 구현되어있음
- 하나는 keras 이용, 또 하나는 tensorflow 2.0 native 그대로 이용
- 앞으로는 계속 keras를 이용할 것이지만 학습목적으로 tf 2.0을 직접 구현해보자

## Import modules

In [1]:
import tensorflow as tf

## 하이퍼파라미터 정의

In [45]:
EPOCHS = 10

## 네트워크 구조 정의

In [46]:
class MyModel(tf.keras.Model): # keras Model을 상속해서 사용
    def __init__(self):
        super().__init__() 
        # 지금 MyModel이라는 calss정의할 때 tf.keras.Model을 상속해서 만들었지
        # init 함수는 tf.keras.Model(상속받은 매개변수)의 init함수를 먼저 call해준다는 뜻
        
        # 우리가 사용하게 될 layer들 정의
        
        # 28x28 MNIST data를 flatten으로 일자로 펴준(벡터 형태) 다음 DNN으로 학습
        self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
        # 1번째 hidden layer 32개, relu이용 (sigmoid, tanh 등도 가능)
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        
        # 출력 layer 정의, 출력은 10개 해야겠지 (MNIST는 손글씨 0~9 10개를 구분할 것이기 때문에)
        # activation function은 정해져있지. multi classificiaton을 위해선 'softmax'사용한다고 배웠었지
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')
        
    def __call__(self, x, training=None, mask=None):
        # __call__은 x라는 입력받아서 출력까지 어떻게 될 것인지 정의
        # training 매개변수는 training할 때랑 test할 때랑 서로 다르게 동작해야하는 알고리즘이 종종 있어
        # ex) batch normalization같은 경우, call이 될 때 training 상태이지 test 상태인지 알기 위해 입력받음
        # mask 매개변수는 mask를 정해줘서 어느 부분에선 동작하고 동작하지않을지 정해줘(지금은 사용하지않아)
        
        # numpy로 구현할 때처럼 다 통과시켜줘
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## 학습 함수 구현

In [47]:
# gradient descent 사용

@tf.function 
# tf.function이라는 decorater(@)를 사용하게되면 이게 쓰여진 함수같은 경우 일반적인 python 함수가 아니라 
# autograph를 통해 이 연산들이 모두 tf 연산으로 바뀌어서 더 최적화된 형태로 컴파일

# train을 한 번 하는 step 구현
# model, 입력 images, 정답 labels, 손실함수 loss_obj, optimizer을 뭘로 할건지
# train_loss, train_accuracy를 학습하면서 요약해줘야함
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    # backpropagation을 통해 gradient를 구해야하는데 tf 2.0에서는 이렇게 구현
    
    # forward 연산 : 미리 gradient를 구할 것이라는 것을 알려줘서 model에 중간값들 다 저장하게 해
    with tf.GradientTape() as tape: # 이 안에서 일어나는 모든 연산에선 gradient 계산해줌
        predictions = model(images) # BatchSize x 10(classes), 입력 image를 model에 넣어 출력값 얻게되면
        loss = loss_object(labels, predictions) # loss_obj통해 label이랑 출력값 비교
        # predictions은 softmax로 나오기때문에 한번에 학습할 batch 개수(ex.32개 image 들어왔다면 32x10 형태로, 10개의 값들 나올 확률 다 더하면 1이 되는 normalization형태로 나오게 됨)
    
    # tape.gradient()이용해서 tape에 저장된 걸 이용해 gradient 구해줘
    gradients = tape.gradient(loss, model.trainable_variables) # loss를 모든 trainable_variables로 미분해서 gradient 계산
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # gradients와 trainable_variables을 같이 iteration해줄 수 있는 개체 넘겨줘(첫번째 variables은 첫번째 gradient를, 2번째, 3번째 등등)
    
    # 학습과정 summary 해주기 위해 
    train_loss(loss) # loss를 누적해서 update
    train_accuracy(labels, predictions) # accuracy 구함

## 테스트 함수 구현

In [48]:
@tf.function
# train과 같지만 test할 땐 optimizer는 필요하지않겠지
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images) # BatchSize x 10(classes)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## 데이터 불러오기

In [49]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
# 값의 범위가 0~255처럼 클 때 normalization하는 것 잊지 말아야 함
# 나눠주는 이유는 x_train과 x_test는 영상이고 0~255의 숫자(integer)로 저장되어있기 때문에 float형태로 바꿔줘야하기때문

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
# tensor가 아니라 numpy여도 돼
# shuffle(1024) : 한 번에 1024개씩 보고 섞을 거라는 뜻

## 모델 생성

In [50]:
model = MyModel() # __init__함수에 아무런 변수도 받지않기때문에 그냥 () 이렇게 불러줘

## 손실함수 및 최적화 알고리즘 정의

In [51]:
# y_train이 어떻게 정의가 되어있느냐에 따라 달라짐
# print(y_train) ; [5 0 4 ... 5 6 8]
# print(y_train.shape) ; (60000,) 6만개 데이터가 있고 각각 0~9까지의 label들이 들어있음
# [5 0 4 ... 5 6 8] 이렇게 되어있다는 건 one-hot vector가 아니라 sparse representation으로 되어있다는 거겠지
# 즉, 0~9까지 class가 0~9라는 숫자로 들어있다는 뜻
# 위 정보 이용해서 loss function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# 이 loss_object는 2가지 입력(label, prediction)을 받게 되는데 prediction같은 경우는 Batchsize x 10(classes)로, batchsize에다가 class개수만큼 크기가 돼서 각각의 확률을 batch별로 가지게 됨
# 근데 label같은 경우 x 10(classes)가 없고 batchsize 크기만 갖는데, spare representation이기 때문에 0~9 class 개수만큼 label을 표현하기 때문
# 예를들어 만약 label이 Batchsize x 10(classes)이고 10이 전부 0이고 하나만 1인 one-hot encoding이었다면 Sparse가 없어지고 그냥 CategoricalCrossentropy()가 되겠지

optimizer = tf.keras.optimizers.Adam() # 고민할 게 없으면 그냥 Adam이 제일 좋다고 말했었지

## 성능 지표 정의

In [52]:
# train_loss, train_accuracy, test_loss, test_accuracy를 받아오는 object들

train_loss = tf.keras.metrics.Mean(name='train_loss') # 위에서 loss를 계속 담아줘서 평균만 구하면 돼
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 마치 loss_object처럼 labels와 predictions을 입력으로 받고있지. labels이 sparse한 게 들어간다는 걸 알려줘야해
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 구현

In [53]:
for epoch in range(EPOCHS):
    for images, labels in train_ds : # batchsize=32로 해줬기때문에 자동적으로 for문 돌면서 32개로 나눠진 images와 labels이 나오게 됨
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
   
    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
          
    # 한 epoch를 돌리고 나면 한 epoch를 학습한 것에 대해 summary를 해서 print해주자(학습이 어떻게 되고있는지 확인 차)
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch +1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
    # for문 돌면서 loss과 accuracy들이 계속해서 call이 되는데, 그 다음에 평균내고 accuracy 내던게 누적이 돼서 결과를 내
    # 그래서 reset_state()를 통해 state들을 초기화해줘서 매 epoch마다 새로운 결과를 다시 얻어낼 수 있게 해줘야 해
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.28755074739456177, Accuracy: 91.24833679199219, Test Loss: 0.1536078304052353, Test Accuracy: 95.3499984741211
Epoch 2, Loss: 0.14003601670265198, Accuracy: 95.74833679199219, Test Loss: 0.1270275115966797, Test Accuracy: 96.06000518798828
Epoch 3, Loss: 0.10794774442911148, Accuracy: 96.66999816894531, Test Loss: 0.13054312765598297, Test Accuracy: 96.16000366210938
Epoch 4, Loss: 0.0913841500878334, Accuracy: 97.18333435058594, Test Loss: 0.11849772930145264, Test Accuracy: 96.5200042

In [ ]:
# loss 점점 감소하고 Accuracy 점점 증가하고있지
# Test loss는 직접 목적으로으로 하는게 아니기때문에 감소할 수도 있고 증가할 수도 있음